<table style="width: 100%;" id="nb-header>">
        <tr style="background-color: transparent;"><td>
            <img src="https://d8a-88.github.io/assets/images/blue_text.png" width="250px" style="margin-left: 0;" />
        </td><td>
            <p style="text-align: right; font-size: 10pt;"><strong>Economic Models</strong>, Fall 2020<br>
                Dr. Eric Van Dusen<br>
            Notebook by Andrei Caprau<br>
            </p></td></tr>
    </table>

# Lecture 11: Econometrics

## What is Econometrics?

One way to think about Econometrics is that it is the closest economists can usually get to doing experiments in the same way experiments are done in medicine and other fields involving people. The idealized experiment is one where people are randomly assigned to either the treatment or control group such that the participants *and* the researchers don't know which group each person is in.

By randomly assigning a large sample of participants to two different groups, we can be fairly confident that on average the two groups are identical in their attributes, apart from the treatment that only one group receives.

By having a double-blind experiment, we can be confident that the participants and researchers aren't conciously or subconciously (placebo effect) affecting the results of the treatment group.

Because of this setup, we can look at the difference in outcomes of the two groups and be fairly confident that any differences are due to the treatment and nothing else.

But it's usually impossible or unethical to perform an ideal experiment in economics. Imagine trying to examine the effect of years of schooling on future earnings. Can you force people into a treatment group with more years of schooling and a control group with less? Is it sufficient to just collect a sample of people and compare the earnings of people with high schooling to those with low schooling?

How can we answer the above question and others like it? Econometrics.

Econometrics is a vast field and today we will just focus on the basics of something called regression.

## Data 8 Review of Regression

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import warnings
warnings.simplefilter("ignore")
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import patches
from datascience import *
%matplotlib inline
from sklearn.metrics import mean_squared_error
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

Suppose we have some data that look like this:

![title](figure1.png)

Recall from Data 8 that $x$ and $y$ above have some **correlation coefficient** $r$, which is a measure of the strength of the linear relationship between the two variables.

It looks like there is some positive linear association between $x$ and $y$ such that larger values of $x$ correspond to larger values of $y$. We therefore expect $r$ to be some positive number between 0 and 1, but not exactly 0 or exactly 1.

First, let's convert the data to standard units. Below we construct a function that does this.

In [ ]:
def standard_units(array):
    return (array - np.mean(array)) / np.std(array)

In [ ]:
# Hide this.
np.random.seed(42)
x = np.random.uniform(0, 10, 100)
noise = np.random.randn(100) * 4
y = 1.5 * x + noise

In [ ]:
x_standard = standard_units(x)
y_standard = standard_units(y)

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(x_standard, y_standard)
plt.xlabel('$x$_standard')
plt.ylabel('$y$_standard');

The plot looks the same as before, except now the axes are scaled such that we measure $x$ and $y$ in standard units. Now recall that $r$ is calculated as the average of the product of two variables, when the variables are measured in standard units. Below we define a function that calculates $r$, assuming that the inputs have already been converted to standard units.

In [ ]:
def correlation(array1, array2):
    return np.mean(array1 * array2)

What is the correlation between these two variables?

In [ ]:
correlation(x_standard, y_standard)

Recall from Data 8 that we use $r$ to form a line called the *regression line*, which makes predictions for $y$ given some $x$. Our prediction for $y$ in standard units is $r \cdot x$. If we want to fit this regression line in the original units, recall that the slope of this line is given by

$$
\text{slope} = r \cdot \dfrac{\hat{\sigma}_y}{\hat{\sigma}_x}
$$

and the intercept is given by

$$
\text{intercept} = \hat{\mu}_y - \text{slope} \cdot \hat{\mu}_x
$$

Below we plot this line.

In [ ]:
r = correlation(x_standard, y_standard)
slope = r * np.std(y) / np.std(x)
intercept = np.mean(y) - slope * np.mean(x)
plt.figure(figsize=(8,6))
plt.scatter(x, y)
plt.plot(np.linspace(0, 10), slope * np.linspace(0, 10) + intercept, color='tab:orange')
plt.xlabel('$x$')
plt.ylabel('$y$');

Let's take a closer look at the slope we found.

In [ ]:
print('Slope: ', slope)

To generate the data above, what I did is I started with some range of $x$ values, and generated $y$ as a linear function of $x$ with some random noise added in. Take a look:

In [ ]:
np.random.seed(42)
x = np.random.uniform(0, 10, 100)
noise = np.random.randn(100) * 4
y = 1.5 * x + noise

Notice how I defined $y$:

$$
y = 1.5 \cdot x + u
$$

where $u$ is some random noise whose average is 0. So, while there is some randomness to the data, on average the "true" slope of the relationship is 1.5. Yet we predicted it to be roughly 1.3!

This highlights the following fact: Suppose we have some random data that we believe has a linear relationship. The least-squares slope we generate from the data is an *estimate* of the "true" slope of that data. Because of this, the estimated slope is a random variable that depends on the data we happen to have.

To highlight this fact, let's repeat the procedure above but with a different [random seed](https://en.wikipedia.org/wiki/Random_seed), in order to get data with the same underlying relationship but different values.

In [ ]:
np.random.seed(189)
x = np.random.uniform(0, 10, 100)
noise = np.random.randn(100) * 4
y = 1.5 * x + noise

r = correlation(x_standard, y_standard)
slope = r * np.std(y) / np.std(x)
intercept = np.mean(y) - slope * np.mean(x)
plt.figure(figsize=(8,6))
plt.scatter(x, y)
plt.plot(np.linspace(0, 10), slope * np.linspace(0, 10) + intercept, color='tab:orange')
plt.xlabel('$x$')
plt.ylabel('$y$');
print('Slope: ', slope)

Now the estimated slope is roughly 1.6, even though the underlying data was still generated using a slope of 1.5. This is a very important concept that we will revisit soon.

Keep in mind, however, that correlation in data *does not* imply causation. In this example we know the true causal relationship between $x$ and $y$ because we defined it ourselves. However, when using real data you do not see the "true" relation and thus cannot conclude causality from correlation. It could simply be that both your variables depend on an unseen third variable and have no causal effect on one another. Or even worse, while unlikely it could be the case that slight linear trends in two variables is a complete coincidence.

### RMSE

In [ ]:
d = Table().with_columns(
    'x', make_array(0,  1,  2,  3,  4),
    'y', make_array(1, .5, -1,  2, -3))

def rmse(target, pred):
    return np.sqrt(mean_squared_error(target, pred))

def plot_line_and_errors(slope, intercept):
    print("RMSE:", rmse(slope * d.column('x') + intercept, d.column('y')))
    plt.figure(figsize=(5,5))
    points = make_array(-2, 7)
    p = plt.plot(points, slope*points + intercept, color='orange', label='Proposed line')
    ax = p[0].axes
    
    predicted_ys = slope*d.column('x') + intercept
    diffs = predicted_ys - d.column('y')
    for i in np.arange(d.num_rows):
        x = d.column('x').item(i)
        y = d.column('y').item(i)
        diff = diffs.item(i)
        
        if diff > 0:
            bottom_left_x = x
            bottom_left_y = y
        else:
            bottom_left_x = x + diff
            bottom_left_y = y + diff
        
        ax.add_patch(patches.Rectangle(make_array(bottom_left_x, bottom_left_y), abs(diff), abs(diff), color='red', alpha=.3, label=('Squared error' if i == 0 else None)))
        plt.plot(make_array(x, x), make_array(y, y + diff), color='red', alpha=.6, label=('Error' if i == 0 else None))
    
    plt.scatter(d.column('x'), d.column('y'), color='blue', label='Points')
    
    plt.xlim(-4, 8)
    plt.ylim(-6, 6)
    plt.gca().set_aspect('equal', adjustable='box')
    
    plt.legend(bbox_to_anchor=(1.8, .8))
    plt.show()

interact(plot_line_and_errors, slope=widgets.FloatSlider(min=-4, max=4, step=.1), intercept=widgets.FloatSlider(min=-4, max=4, step=.1));

## Now Back to Economics

We know from above that if we have some (well-behaved) data, we can fit a least-squares line onto that data. That line can have two purposes:

* Of particular interest to data scientists is the line's ability to predict values of $y$ for new values of $x$ that we didn't see before.

* Of particular interest to economists is the line's ability to estimate the "true" underlying slope of the data.

This is why regression is such a powerful tool and forms the backbone of econometrics. If we believe that our data satisfy certain assumptions (which we won't explore too much this lecture), then we can use the slope of the regression line to estimate the "true" relation between the variables in question and learn more about the world we live in.

In econometrics, we usually write the "true" underlying linear relationship as follows:

$$
y = \alpha + \beta \cdot x + \varepsilon
$$

where $y$ and $x$ are values for any arbitrary point, $\alpha$ is the intercept, $\beta$ is the slope, and $\varepsilon$ is some noise.

This is entirely analogous to the code from earlier that determined the true linear relationship between the data:

```python
y = 1.5 * x + noise
```

Here, $\beta = 1.5$, $\alpha = 0$, and $\varepsilon = \text{noise}$.

When we fit a regression line onto the data, we express the line as:

$$
\hat{y} = \hat{\alpha} + \hat{\beta} \cdot x
$$

Here, we put hats over the slope and intercept terms because they are *estimates* of the true slope and intercept terms. Similarly, we put a hat over $y$ because this is the $y$ value that the regression line predicts.

Notice how the noise term $\varepsilon$ does not appear in the expression for the regression line. This is because the noise term is a random variable that has no relation with $x$, and is thus impossible to predict from the data. Furthermore, the noise term has a mean value of 0, so on average we actually don't expect the noise term to have any impact on the underlying trends of the data.

For the Data 8 demonstration above, we forced these conditions to be true. However, with real data these are assumptions that we have to make, and is something that econometricians spend a lot of time thinking about.

## Example: Years of Schooling and Earnings

Consider a case where we want to study how years of schooling relate to a person's earnings. This should be of particular interest to college students.

Below we import a dataset that has the hourly wage, years of schooling, and other information on thousands of people sampled in the March 2012 Current Population Survey.

In [ ]:
cps = Table.read_table('cps.csv')
cps

We want to consider a person's wage and years of schooling. But first...

### Brief Detour on Wage and Similar Variables

Wage is an interesting variable because it is something that tends to vary by a proportion rather than an absolute amount.

To highlight this fact, let's actually consider how the variable GDP behaves. GDP tends to grow by a certain percent each year; no one is particularly interested in how *much* GDP changes from one year to the next, or from one country to another, but rather by *what percent* it changes.

If you recall from the finance week, this behaves similarly to compound interest. If you were to plot GDP over time for a country, it might look something like this:

In [ ]:
GDPs = []
GDPs.append(100)

for _ in range(99):
    GDPs.append(GDPs[-1] * 1.05)
    
plt.figure(figsize=(8,6))
plt.plot(np.arange(100), GDPs)
plt.xlabel('Years')
plt.ylabel('GDP')
plt.title('GDP Over Time');

The starting value for GDP is 100, and GDP grows by 5 percent each year. Look at how how much GDP grows in the later years!

While this phenomenon is impressive, it is misleading. At surface level, it seems to imply that something different happened at around year 50 or so that caused GDP to increase considerably in subsequent years. We know that this isn't true, and that this is just a consequence of exponential growth.

To counter this effect, for variables that tend to vary from one observation to the next by proportions rather than absolute amounts, we take the natural log of these variables. Let's do that for GDP:

In [ ]:
ln_GDPs = np.log(GDPs)

plt.figure(figsize=(8,6))
plt.plot(np.arange(100), ln_GDPs)
plt.xlabel('Years')
plt.ylabel('GDP')
plt.title('GDP Over Time');

We've now uncovered a linear relationship between years and GDP! You can interpret the slope of this line as the approximate *percent change* in GDP for an increase in one year. To verify:

In [ ]:
print('Slope between years 0 and 1: ', ln_GDPs[1] - ln_GDPs[0])

It turns out that wage is another one of these variables, and so when we do studies on wage we usually take the natural log of wage instead.

Below we plot log wage and years of schooling for the CPS data.

In [ ]:
educ = cps.column('educ')
logwage = cps.column('logwage')

plt.figure(figsize=(8,6))
plt.scatter(educ, logwage)
plt.xlabel('Years of Education')
plt.ylabel('Log Wage')
plt.title('Log Wage vs. Years of Education');

Now let's fit a least-squares regression line onto this data. First, we'll do it manually in the Data 8 style above.

In [ ]:
educ_standard = standard_units(educ)
logwage_standard = standard_units(logwage)

r = correlation(logwage_standard, educ_standard)
slope = r * np.std(logwage) / np.std(educ)
intercept = np.mean(logwage) - slope * np.mean(educ)

plt.figure(figsize=(8,6))
plt.scatter(educ, logwage)
plt.plot(np.linspace(0, 20), slope * np.linspace(0, 20) + intercept, color='tab:orange')
plt.xlabel('Years of Education')
plt.ylabel('Log Wage')
plt.title('Log Wage vs. Years of Education');
print('Slope: ', slope)
print('Intercept: ', intercept)

It turns out that there are statistical packages that actually do all this work for you. From now on we'll use this instead. Below we verify that we get the same results using the package `statsmodels`, which is the main statistics package in Python and which we have imported as `sm`.

To do single variable regression in Python, we use the `statsmodels` package, which has a very simple flow:

```python
X = tbl.column(feature)                     # Separate explanatory variable
y = tbl.column(target)                      # Separate outcome variable
model = sm.OLS(y, sm.add_constant(X))       # Initialize the OLS regression model
result = model.fit()                        # Fit the regression model and save it to a variable
result.summary()                            # Display a summary of results
```

Note: In order for `statsmodels` to give us an intercept term, we must add a column of all 1's to the data. This is typical in econometrics and you should essentially always have this constant column.

In [ ]:
sm.add_constant(educ)

In [ ]:
model = sm.OLS(logwage, sm.add_constant(educ)).fit()
model.summary()

As we expect, the numbers are the same.

So from the very simple and straight-forward model above, it seems that we estimate a slope of roughly 0.1, meaning we might expect that a one-year increase in schooling is associated with a 10 percent increase in wage, on average.

We can also see that we have a non-zero intercept term. We should be careful how we interpret this term; from a strictly mathematical point of view, the intercept represents the expected value of $y$ (in this case log wage) when $x = 0$.

However, in economics sometimes it makes no sense for $x$ to be 0, and so we cannot use the above interpretation. We won't go into detail this lecture, but regardless of whether the intercept is interpretable, we almost always want to include it.

### Uncertainty in $\hat{\beta}$

We mentioned earlier that the slope we estimate from regression is exactly that: an estimate of the "true" underlying slope. Because of this, the estimate $\hat{\beta}_1$ is a random variable that depends on the underlying data.

We assume there is some true linear relation between log wage and years of schooling,

$$
\text{logwage} = \alpha + \beta \cdot \text{years of schooling} + \varepsilon
$$

and we try to estimate $\alpha$ and $\beta$.

If our data are "well-behaved", then even though there is uncertainty in our estimate $\hat{\beta}$, on average $\hat{\beta}$ will be $\beta$; that is to say that the expectation of $\hat{\beta}$ is $\beta$.

Additionally, if our data are "well-behaved", then $\hat{\beta}$ has some normal distribution with mean $\beta$. We won't worry too much about what assumptions need to be satisfied to make the data "well-behaved".

You can think of each person as an observation of these variables, and using a sample of people we can estimate the relationship between the two variables. However, due to the noise term and the fact that we only have a finite sample of people, the true relationship is always hidden from us, and we can only hope to get better estimates by designing better experiments and sampling more people.

Let's try to get an idea of how "certain" we can be of our estimate $\hat{\beta}$. We'll do this in classic Data 8 style: bootstrapping.

Using our existing sample data, we'll create new samples by bootstrapping from the existing data. Then, for each sample, we'll fit a line, and keep the slope of that line in a list with all of the other slopes. Then, we'll find the standard deviation of that list of slopes.

In [ ]:
slopes = make_array()
educ_logwage = cps.select("educ", "logwage")
np.random.seed(42)

for i in np.arange(200):
    educ_logwage_sample = educ_logwage.sample()
    y = educ_logwage_sample.column("logwage")
    X = educ_logwage_sample.column("educ")
    model = sm.OLS(y, sm.add_constant(X)).fit()
    slopes  = np.append(model.params[1], slopes)
Table().with_columns("Slopes", slopes).hist()    
print('Standard dev. of bootstrapped slopes: ', np.std(slopes))

We now have an estimate of the standard deviation of $\hat{\beta}$. This is called the *standard error* of $\hat{\beta}$. `statsmodels` actually already does this for us. Look at the column called "std err" below.

In [ ]:
model = sm.OLS(logwage, sm.add_constant(educ)).fit()
model.summary()

In the summary table, `statsmodels` rounds numbers, so let's see an expanded form of the standard error.

In [ ]:
print('SE of beta 1: ', model.bse[1])

Our bootstrapped approximation standard error of 0.00159 is pretty close to the true standard error of 0.00144. `statsmodels` actually uses a precise mathematical formula for finding the standard error whereas we tried to find this value through simulation, but the idea behind the standard error is the same.

Armed with a standard error, we can now form a 95% confidence interval and perform a test of significance to see if $\hat{\beta}$ is significantly different from 0.

In [ ]:
# Using our resampled slopes
lower_bound = percentile(2.5, slopes)
upper_bound = percentile(97.5, slopes)
print('95% confidence interval: [{}, {}]'.format(lower_bound, upper_bound))

In [ ]:
# Using the given standard deviation in sample slopes from statsmodels
lower_bound = model.params[1] - 1.96 * model.bse[1]
upper_bound = model.params[1] + 1.96 * model.bse[1]
print('95% confidence interval: [{}, {}]'.format(lower_bound, upper_bound))

Ah, it appears `statsmodels` already beat us to it.

In [ ]:
model = sm.OLS(logwage, sm.add_constant(educ)).fit()
model.summary()

The 95% confidence interval does not contain 0, and so $\beta$ is unlikely to be 0.

Earlier we said that $\hat{\beta}$ has some normal distribution with mean $\beta$ if certain assumptions are satisfied. We now can see that the standard deviation of that normal distribution is the standard error of $\hat{\beta}$.

We can also use this to test a null hypothesis that $\beta = 0$. To do so, construct a t-statistic (which `statsmodels` does for you) that indicates how many standard deviations away $\hat{\beta}$ is from 0, assuming that the distribution of $\hat{\beta}$ is in fact centered at 0.

In [ ]:
model = sm.OLS(logwage, sm.add_constant(educ)).fit()
model.summary()

We can see that $\hat{\beta}$ is 74 standard deviations away from the null hypothesis mean of 0, which is an enormous number. How likely do you think it is to draw a random number roughly 74 standard deviations away from the mean, assuming a standard normal distribution?

Essentially 0. This is strong evidence that the mean of the distribution (the mean of $\hat{\beta}$ is the true value $\beta$) is not 0.

## Regression with a Binary Variable

A binary variable is a variable that takes on the value of 1 if some condition is true, and 0 otherwise. These are also called dummy variables or indicator variables.

It might sound strange at first, but you can actually perform regression of a variable like log earnings onto a binary variable.

Let's import a different dataset that has the following features. Some will be useful to us later.

In [ ]:
nlsy = Table.read_table('nlsy_cleaned_small.csv')
nlsy

Now let's visualize log earnings vs. the binary variable corresponding to whether or not an observation went to college.

In [ ]:
coll = nlsy.column('college')
logearn = nlsy.column('log_earn_1999')

plt.figure(figsize=(8,6))
plt.scatter(coll, logearn)
plt.xlabel('College')
plt.ylabel('Log Earnings')
plt.title('Log Earnings vs. College Completion')
plt.xticks([0,1]);

In [ ]:
no_college = nlsy.where('college', 0).column("log_earn_1999")
has_college = nlsy.where('college', 1).column("log_earn_1999")

plt.figure(figsize=(8,6))
plt.xlabel('College')
plt.ylabel('Log Earnings')
plt.title('Log Earnings vs. College Completion')
plt.violinplot(no_college, positions = [0], points=20, widths=0.3, showmeans=True, showextrema=True, showmedians=False)
plt.violinplot(has_college, positions = [1], points=20, widths=0.3, showmeans=True, showextrema=True, showmedians=False);

Now let's fit a regression model:

In [ ]:
model = sm.OLS(logearn, sm.add_constant(coll)).fit()
model.summary()

Wow! This regression would imply that we expect, on average, observations who went to college to have 70% higher earnings than those who did not go to college. Let's now plot this line on the data:

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(coll, logearn)
plt.plot(np.linspace(0, 1), model.params[1] * np.linspace(0, 1) + model.params[0], color='tab:orange')
plt.xlabel('College')
plt.ylabel('Log Earnings')
plt.title('Log Earnings vs. College Completion')
plt.xticks([0,1]);

In [ ]:
no_college = nlsy.where('college', 0).column("log_earn_1999")
has_college = nlsy.where('college', 1).column("log_earn_1999")

plt.figure(figsize=(8,6))
plt.plot(np.linspace(0, 1), model.params[1] * np.linspace(0, 1) + model.params[0], color='tab:green')
plt.xlabel('College')
plt.ylabel('Log Earnings')
plt.title('Log Earnings vs. College Completion')
plt.violinplot(no_college, positions = [0], points=20, widths=0.3, showmeans=True, showextrema=True, showmedians=False)
plt.violinplot(has_college, positions = [1], points=20, widths=0.3, showmeans=True, showextrema=True, showmedians=False);

When we perform a simple regression onto just a dummy variable, it is an important fact that $\hat{\alpha}$ is the mean value of $y$ for all observations in the sample where $x = 0$, and $\hat{\beta}$ is the difference between the mean value of $y$ for observations in the sample where $x = 1$ and observations where $x = 0$.

Proving this claim is beyond our scope this week, but let's verify it with our data:

In [ ]:
avg_logearn_coll = np.mean(logearn[coll == 1])
avg_logearn_nocoll = np.mean(logearn[coll == 0])

print('Avg logearn for coll = 1: ', avg_logearn_coll)
print('Avg logearn for coll = 0: ', avg_logearn_nocoll)
print('Difference between the two: ', avg_logearn_coll - avg_logearn_nocoll)

In [ ]:
print('Intercept: ', model.params[0])
print('Slope: ', model.params[1])

## Multivariable Regression and Bias

Our procedure earlier showed that we expect to see roughly a 70% increase in earnings in people who went to college vs. people who did not go to college. Does this imply that your decision to go to college was worthwhile, and now you can expect to have roughly 70% higher earnings compared to the version of you who did not go to college?

Let's go back to our discussion of experiments from earlier. In an ideal experiment, we would want a good sample of people who are about to graduate high school, and then randomly assign them to either a treatment group that gets sent to college, and a control group that does not. If you are in the treatment group you *must* go to college, and if you are in the control group you *cannot* go to college.

Since the decision to go to college in this case is completely random, we can safely assume that the treatment and control groups are on average identical in attributes, apart from college attendance. We can therefore compare their log earnings in the future to see the effect of going to college.

Clearly this experiment is impossible to perform. We cannot randomly assign people to go to college.

What's different between this ideal experiment and our regression from earlier? What's the issue with comparing the differences in log earnings for people in our sample who happened to go to college and those who did not?

In our sample, the treatment (went to college) and control (did not go to college) groups are not identical in every way except for college! People aren't randomly assigned college, they *choose* to go to college.

The factors that cause someone to go to college are complex and lead to differences between people who chose to go to college and those who did not.

When we perform regression on the variable "college", since the groups in the sample are different, not only are we capturing the effect of going to college, but we are also capturing the effects of everything else that is different about the two groups that also affects earnings.

Imagine another variable that captures the wealth of your family. College can be very expensive, so it might be the case that the wealthier your family is, the more likely you are to go to college.

Also, it's easy to imagine that the wealthier your family is, the wealthier you are likely to be in the future.

This implies that the group of people in the sample who went to college will tend to be wealthier than the group that did not. Also, the group of people who went to college is expected to earn more not necessarily because they went to college, but simply because they are wealthier.

Therefore, when we do regression to measure the differences in earnings between people who went to college and those who did not, we also capture differences in earnings between people who grew up wealthier and those who did not.

Because of this, we *over-estimate* the effect of going to college. $\hat{\beta}$ captures the average observed benefit of going to college *and* being wealthier, but we're only interested in college. This is called *omitted variable bias*. Family wealth is an omitted variable in this regression and it's causing our results to be biased.

Let's think of another example of omitted variable bias. In the NLSY dataset, there is a variable called "AFQT". AFQT is a score on a particular standardized test that all people in the sample took.

Let's use AFQT as a proxy measurement for the abstract idea of academic capability. While a standardized test is by no means the sole indication of someone's ability or intelligence, let's ignore that very complicated issue for now and assume that AFQT does an ok job at capturing this ability variable that is otherwise very difficult to measure.

Is there omitted variable bias from AFQT? Almost certainly. It seems fair to believe that people who choose to go to college are on average more academically-capable, and it also seems fair to say that on average we expect more capable people to earn more.

Therefore, $\hat{\beta}$ above might be capturing the effects of being more capable, along with the effects of going to college.

How can we fix this issue? Multivariable regression.

### Multivariable Regression

So far we have only been regressing outcome variable $y$ onto one explanatory variable $x$. To find the regression line, we choose $\hat{\alpha}$ and $\hat{\beta}$ that minimize the mean squared error.

But what if we believe that $y$ is actually determined by two variables, $x_1$ and $x_2$? Specifically, what if this is the "true" model:

$$
y = \alpha + \beta_1 x_{1} + \beta_2 x_{2} + \epsilon
$$

and we would like to estimate the following:

$$
\hat{y} = \hat{\alpha} + \hat{\beta}_1 x_{1} + \hat{\beta}_2 x_{2}
$$

Now our challenge is choosing $\hat{\alpha}$, $\hat{\beta}_1$, *and* $\hat{\beta}_2$ that minimize the mean squared error.

In [ ]:
def rmse(target, pred):
    return np.sqrt(mean_squared_error(target, pred)) 

def to_minimize(intercept, beta_1, beta_2):
    predictions = intercept + beta_1 * nlsy.column('college') + beta_2 * nlsy.column("AFQT")
    actual = nlsy.column('log_earn_1999')
    return rmse(predictions, actual)

minimize(to_minimize)

To do multivariable regression in Python using the `statsmodels` package, the procedure is largely the same with some minor differences. Instead of only selecting 1 column for $x$, we need to specify that $x$ is in fact multiple variables by using `select` to select out the relevant columns. Note that here, we take out the values of each by adding `.values` to the end. This is because `statsmodels` only knows how to work with NumPy arrays, not tables.

```python
X = tbl.select(features).values             # Separate features (explanatory and control variables) 
y = tbl.column(target)                      # Separate outcome variable
model = sm.OLS(y, sm.add_constant(X))       # Initialize the OLS regression model
result = model.fit()                        # Fit the regression model and save it to a variable
result.summary()                            # Display a summary of results
```

In [ ]:
y = nlsy.column('log_earn_1999')
X = nlsy.select('college', 'AFQT').values.astype(float)

model = sm.OLS(y, sm.add_constant(X)).fit()
model.summary()

Here $\hat{\beta}_1$ is 0.43, compared to 0.72 from the earlier "biased" regression.

That's huge! This implies that when we control for a person's ability (i.e. we get rid of that source of bias), we only see that on average going to college is associated with a 43% increase in earnings instead of 72%.

Furthermore, looking at the 95% confidence interval for $\hat{\beta}_2$, we see that it does not contain 0, which would imply that AFQT score has a strong non-zero association with earnings.

These observations validate our claim that AFQT was probably an omitted variable causing $\hat{\beta}_1$ to be biased.

When interpreting $\hat{\beta}$ coefficients, you should always be mindful of potential sources of bias that could make your coefficients misleading and not useful from an econometric context.

### Visualizing Multivariable Regression

In [ ]:
ax = plt.figure(figsize=(8,8)).add_subplot(111, projection='3d')
ax.scatter(nlsy.column("AFQT"), 
           nlsy.column("college"), 
           nlsy.column('log_earn_1999'))
plt.xlabel("AFQT")
plt.ylabel("college");

In [ ]:
X,Y = np.meshgrid(np.arange(0,100,1), np.arange(0,1,0.01))
Z = 0.0084 * X + 0.4301 * Y + 9.9563
ax = plt.figure(figsize=(8,8)).add_subplot(111, projection='3d')
ax.scatter(nlsy.column("AFQT"), 
           nlsy.column("college"), 
           nlsy.column('log_earn_1999'))
ax.plot_surface(X, Y, Z, alpha=0.5)
plt.xlabel("AFQT")
plt.ylabel("college");

In [ ]:
X,Y = np.meshgrid(np.arange(0,1,0.01), np.arange(0,100,1))
Z = 0.4301 * X + 0.0084 * Y + 9.9563
ax = plt.figure(figsize=(8,8)).add_subplot(111, projection='3d')
ax.scatter(nlsy.column("college"), 
           nlsy.column("AFQT"), 
           nlsy.column('log_earn_1999'))
ax.plot_surface(X, Y, Z, alpha=0.1)
plt.ylabel("AFQT")
plt.xlabel("college");

## Colinearity and Dummy Variables

When we do regression onto the variable "college", why don't we also include a variable that measures not going to college? In other words, why don't we regress on college and the opposite of college so that way we can get an estimate of the average earnings of college-goers and non-college-goers. Why do we do this roundabout way of using the intercept term and a difference in means?

Imagine we have a dataset with a variable for college, a variable for not going to college, and the intercept term. The issue with this is that there is now redundant information.

Let's look at just one element of the sample. Let's say this person went to college, so this person's features are the following:
* College = 1
* Not College = 0
* Intercept term = 1

Clearly there is a redundancy here; you can guess one of the variables from the others.

More specifically, by redundancy we mean that *one variable can be written as a linear combination of the other variables*. In fact, there are three different combinations:
* Intercept = College + Not College
* Not College = Intercept - College
* College = Intercept - Not College

These equalities aren't just true for this one person; they actually hold true for any possible person in the sample. This is because of the way we defined "college" and "not college". You can't simultaneously be in both, and so adding them together you get 1, which is just the intercept term.

In general, we have redundancy whenever we have *mutually exclusive* and *exhaustive* dummy variables in combination with an intercept term.
* Mutually exclusive: You cannot be in more than one dummy variable.
* Exhaustive: You must be in at least one dummy variable.

You can see that "college" and "not college" satisfy these conditions. So why is this redundancy an issue?

It becomes ambiguous what the values for $\hat{\alpha}$, $\hat{\beta}_1$, and $\hat{\beta}_2$ should be in the model where we include all three terms:

$$
\text{log earnings} = \hat{\alpha} + \hat{\beta}_1 \text{college} + \hat{\beta}_2 \text{not college}
$$

Consider a case where we expect people who went to college to have log earnings of 10 and those who did not go to college to have log earnings of 8. What values for $\hat{\beta}$ and $\hat{\alpha}$ make sense?

* $\hat{\beta}_1 = 10$
* $\hat{\beta}_2 = 8$
* $\hat{\alpha} = 0$

make sense. These are valid values for $\hat{\beta}$ and $\hat{\alpha}$ that satisfy the condition above. To see why, consider a person with college:

$$
\text{log earnings} = \hat{\alpha} + \hat{\beta}_1 \cdot 1 + \hat{\beta}_2 \cdot 0 \\
\text{log earnings} = \hat{\alpha} + \hat{\beta}_1 \\
\text{log earnings} = 0 + 10 = 10
$$

and a person without college:

$$
\text{log earnings} = \hat{\alpha} + \hat{\beta}_1 \cdot 0 + \hat{\beta}_2 \cdot 1 \\
\text{log earnings} = \hat{\alpha} + \hat{\beta}_2 \\
\text{log earnings} = 0 + 8 = 8
$$

* $\hat{\beta}_1 = 2$
* $\hat{\beta}_2 = 0$
* $\hat{\alpha} = 8$

also make sense. To see why, consider a person with college:

$$
\text{log earnings} = \hat{\alpha} + \hat{\beta}_1 \cdot 1 + \hat{\beta}_2 \cdot 0 \\
\text{log earnings} = \hat{\alpha} + \hat{\beta}_1 \\
\text{log earnings} = 8 + 2 = 10
$$

and a person without college:

$$
\text{log earnings} = \hat{\alpha} + \hat{\beta}_1 \cdot 0 + \hat{\beta}_2 \cdot 1 \\
\text{log earnings} = \hat{\alpha} + \hat{\beta}_2 \\
\text{log earnings} = 8 + 0 = 8
$$

It turns out, there are actually infinitely many solutions for $\hat{\beta}$ that satisfy the condition where people who went to college have mean log earnings of 10 and people who did not go to college have mean log earnings of 8.

This holds true for all situations where you regress on a constant and a set of mutually exclusive and exhaustive dummies. There is no unique solution for $\hat{\beta}$, which is a problem for econometricians who want unique and interpretable coefficients.

In fact, there is mathematical justification for this as well. At some point in the math involved in performing regression, having redundant variables causes a division by 0. This is particularly upsetting for your computer, and it will complain.

So how do we avoid this problem? We deliberately exclude one of the variables. It can technically either be one of the dummy variables or the intercept term, but we usually really want to have an intercept term present in our regression for other reasons. So we usually get rid of one of the dummy variables.

Notice that we implicitly did this earlier. We did not include "not college" in our first regression.